In [1]:
import os
import sys
import numpy as np
import parmed as pmd

from openmm import * 
from openmm.app import * 
from openmm.unit import * 

from sys import stdout

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [3]:
pdb = PDBFile('../../1ubq.pdb')

In [4]:
# forcefield = ForceField('amoeba2018.xml')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [5]:
modeller = Modeller(pdb.topology, pdb.positions)

In [6]:
_ = modeller.addHydrogens(forcefield)

In [7]:
modeller.addSolvent(forcefield, boxSize=Vec3(6, 6, 6)*nanometers, ionicStrength=0.1*molar, positiveIon='K+')

In [8]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME)
integrator = VerletIntegrator(0.001*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=100)
print('Saving...')
# positions = simulation.context.getState(getPositions=True).getPositions()
# PDBFile.writeFile(simulation.topology, positions, open('1ubq_sol.pdb', 'w'))
print('Done')

Saving...
Done


In [9]:
output_traj = 'output.dcd'
report_freq = 1000

simulation.reporters.append(app.DCDReporter(output_traj, report_freq))
simulation.reporters.append(app.StateDataReporter(stdout,
        report_freq, step=True, time=True, speed=True,
        potentialEnergy=True, temperature=True, totalEnergy=True))
simulation.reporters.append(app.CheckpointReporter('checkpnt.chk', report_freq))

In [10]:
simulation.step(1e4)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,1.0000000000000007,-387299.1128701712,-377485.7766909302,55.365093024829044,0
2000,1.9999999999998905,-388846.3628701712,-377487.85349680786,64.08268468474017,197
3000,2.9999999999997806,-389582.9878701712,-377488.92360306164,68.23255424727219,210
4000,3.9999999999996705,-390130.2378701712,-377490.915981693,71.30880052867762,207
5000,5.000000000000004,-390599.4878701712,-377494.580310335,73.93555187347447,211
6000,6.000000000000338,-390850.7378701712,-377493.8055821316,75.35743045448692,208
7000,7.000000000000672,-391014.4878701712,-377494.7932715051,76.27570639084317,210
8000,8.000000000001005,-391095.1128701712,-377499.1860746541,76.70579485342634,209
9000,9.000000000000451,-391142.3628701712,-377495.2095983142,76.9948054993567,211
10000,9.999999999999897,-391333.6128701712,-377494.1101782369,78.08000663191125,212


In [11]:
state = simulation.context.getState(getPositions=True, getVelocities=True, getForces=True,
getEnergy=True, getParameters=True, enforcePeriodicBox=True) 

In [12]:
with open("omm_system.xml", "w") as f:
    system_xml = XmlSerializer.serialize(system)
    f.write(system_xml)

# integrator.xml contains the confiruation for the integrator, RNG seed
with open('omm_integrator.xml', 'w') as f:
    integrator_xml =  XmlSerializer.serialize(integrator)
    f.write(integrator_xml)
# state.xml contains positions, velocities, forces, the barostat
with open('omm_state.xml', 'w') as f:
    f.write( XmlSerializer.serialize(state)) 

In [13]:
with open("omm_top.xml", "w") as f: 
    top_xml = XmlSerializer.serialize(modeller.topology)
    f.write(top_xml)

ValueError: Unsupported object type

In [16]:
system =  XmlSerializer.deserialize(open('omm_system.xml').read())
integrator =  XmlSerializer.deserialize(open('omm_integrator.xml').read())
state =  XmlSerializer.deserialize(open('omm_state.xml').read()) 

In [18]:
simulation = app.Simulation(topology, system, integrator)
simulation.context.setState(state) 